In [130]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset, DatasetDict
import pandas as pd
import torch
import ast

In [131]:
tag_list_path = '../../data/wordlist/tag.txt'
financeNer_path = '../../data/FinanceNewsNER.csv'

# Dataset Prepare

In [ ]:
df_financeNer = pd.read_csv(financeNer_path, index_col=0)
# Parsing ulang kolom yang seharusnya berupa list
df_financeNer['tokens'] = df_financeNer['tokens'].apply(ast.literal_eval)
df_financeNer['ner_tags'] = df_financeNer['ner_tags'].apply(ast.literal_eval)

In [150]:
df_financeNer.head(5)

,tokens,ner_tags
0,"[Tantangan, Ketimpangan, Ekonomi]","[O, B-ECONOMIC_INDICATOR, I-ECONOMIC_INDICATOR]"
1,"[Doktor, ekonomi, dari, UNU-MERIT, Maastricht,...","[O, O, O, O, O, O, O]"
2,"[Alumni, generasi, pertama, beasiswa, LPDP, ma...","[O, O, O, O, O, I-PRD]"
3,"[Pernah, bekerja, di, ASEAN, Secretariat, Indo...","[O, O, O, B-NOR, O, O, O, O, O, I-NOR]"
4,"[Saat, ini, berkiprah, sebagai, akademisi, pen...","[O, O, O, O, O, O, O, O]"


In [ ]:
unique_tags = set(tag for tags in df_financeNer["ner_tags"] for tag in tags)

label_to_id = {"O": 0}
id_to_label = {0: "O"}
current_id = 1

for tag in unique_tags:
    if tag != 'O':
        label_to_id[tag] = current_id
        id_to_label[current_id] = tag
        current_id += 1

In [160]:
df_financeNer['ner_tags'] = [list(map(lambda x: label_to_id.get(x, -1), ner_tag)) for ner_tag in df_financeNer['ner_tags']]

In [162]:
df_train, df_test = train_test_split(df_financeNer, test_size=0.2, random_state=42)

In [163]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

datasets = DatasetDict({
    "train" : train_dataset,
    "test" : test_dataset
})

# Modelling

# IndoBERT

In [164]:
# training_args = TrainingArguments(
#     output_dir="./results",          # Direktori output
#     evaluation_strategy="epoch",    # Evaluasi setiap epoch
#     learning_rate=2e-5,             # Learning rate
#     per_device_train_batch_size=16, # Batch size training
#     per_device_eval_batch_size=16,  # Batch size evaluasi
#     num_train_epochs=3,             # Jumlah epoch
#     weight_decay=0.01,              # Regularisasi L2
#     logging_dir="./logs",           # Direktori logging
#     logging_steps=10,
#     save_steps=500,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy", # Evaluasi berdasarkan akurasi
# )

In [ ]:
# # Fungsi evaluasi
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = torch.argmax(torch.tensor(logits), dim=-1)
#     return {"accuracy": accuracy_score(labels, predictions)}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["test"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

NameError: name 'model' is not defined

### indobenchmark/indobert-base-p1

In [166]:
model1_name = "indobenchmark/indobert-base-p1"
tokenizer_p1 = BertTokenizerFast.from_pretrained(model1_name)
classifier_p1 = BertForTokenClassification.from_pretrained(model1_name, num_labels=len(unique_tags))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [167]:
def tokenize_and_align_labels(examples):
    # Tokenize the inputs with word-level tokenization
    tokenized_inputs = tokenizer_p1(
        examples["tokens"], 
        is_split_into_words=True, 
        truncation=True, 
        padding=True
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        # Get word IDs for the current batch
        word_ids = tokenized_inputs.word_ids(batch_index=i)  
        label_ids = []

        # Align labels with tokens
        for word_id in word_ids:
            if word_id is None:  # Special tokens get -100
                label_ids.append(-100)
            elif word_id != word_ids[word_ids.index(word_id) - 1]:  # First sub-token
                label_ids.append(label[word_id])
            else:  # Subsequent sub-tokens get -100
                label_ids.append(-100)
        
        labels.append(label_ids)

    # Add labels to the tokenized inputs
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [168]:
train_dataset_model1 = train_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2241 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
trainer.train()

### indobenchmark/indobert-base-p2

In [23]:
model2_name = "indobenchmark/indobert-base-p2"
tokenizer_p2 = BertTokenizer.from_pretrained(model2_name)
classifier_p2 = BertForTokenClassification.from_pretrained(model2_name, num_labels=len(label))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
